<a href="https://colab.research.google.com/github/YorkJong/vistock/blob/feature%2Fibd/notebooks/ibd_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Stock Analysis and Ranking with IBD RS Rating, inspired by the Investor's Business Daily (IBD) methodology.

### Install vistock from GitHub (免費版Colab會固定時間清掉安裝的東西，所以重安裝是新連線後最先要做的事)

In [45]:
%pip install "git+https://github.com/YorkJong/vistock.git@feature/ibd"
%pip install requests-cache

  Cloning https://github.com/YorkJong/vistock.git (to revision feature/ibd) to /tmp/pip-req-build-cffqhm3s
  Running command git clone --filter=blob:none --quiet https://github.com/YorkJong/vistock.git /tmp/pip-req-build-cffqhm3s
  Running command git checkout -b feature/ibd --track origin/feature/ibd
  Switched to a new branch 'feature/ibd'
  Branch 'feature/ibd' set up to track remote branch 'feature/ibd' from 'origin'.
  Resolved https://github.com/YorkJong/vistock.git to commit f7d6cb118a1ef577c53cda1b061ae8d36060e365
  Preparing metadata (setup.py) ... done


### Setup and Configuration

In [46]:
# @title Enable DataFrame Formaatter
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [47]:
# @title Install Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [48]:
# @title Initialize Widgets
import ipywidgets as widgets
output = widgets.Output()

In [49]:
# @title Update and Filter DataFrame

def update_tickers_with_names(df, ticker_column, name_separator=','):
    """
    Update ticker codes in a DataFrame with their corresponding stock names.

    Parameters:
    - df: The DataFrame containing ticker codes.
    - ticker_column: The name of the column containing ticker codes.
    - name_separator: Separator used to join names (default is comma for multiple tickers).

    This function updates the specified column with the stock names instead of ticker codes.
    """
    # Iterate over the specified column in the DataFrame
    for index, row in df.iterrows():
        tickers = row[ticker_column].split(name_separator)  # Split the tickers string into a list
        stock_names = [tw.stock_name(ticker) for ticker in tickers]  # Get stock names for each ticker
        # Update the stock names back to the DataFrame
        df.at[index, ticker_column] = name_separator.join(stock_names)  # Join the names back into a string


def remove_ticker_suffix(df, ticker_column):
    """
    Remove the '.TW' or '.TWO' suffix from ticker codes in a DataFrame.

    Parameters:
    - df: The DataFrame containing ticker codes.
    - ticker_column: The name of the column containing ticker codes.
    """
    # Apply string replacement for each ticker in the specified column
    df[ticker_column] = df[ticker_column].str.replace('.TWO', '', regex=False)
    df[ticker_column] = df[ticker_column].str.replace('.TW', '', regex=False)


def filter_increasing_relative_strength(df):
    """
    Filter stocks with increasing Relative Strength over different time periods.

    This function filters the DataFrame to include only those stocks where:
    - Relative Strength is above 100.
    - Relative Strength has increased over the past 1 month, 3 months, and 6 months.
    Optionally, you can add a condition to check if Percentile is above 90.
    """
    return df[
        (df["Relative Strength"] > 100)
        & (df["Relative Strength"] > df["1 Month Ago"])
        & (df["1 Month Ago"] > df["3 Months Ago"])
        & (df["3 Months Ago"] > df["6 Months Ago"])
        # & (df["Percentile"] > 90)  # Uncomment to include Percentile filter
    ]


In [50]:
# @title Rank Function

import os
from datetime import datetime

from vistock import ibd
from vistock import tw
from vistock.stock_indices import get_tickers

def rank(code, period='2y', tickers_getter=get_tickers,
         ref_ticker='^GSPC', out_dir='out'):
    tickers = tickers_getter(code)

    output.clear_output()
    with output:
        rank_stock, rank_indust = ibd.rankings(tickers, period=period,
                                               ref_ticker=ref_ticker)
    if rank_stock.empty or rank_indust.empty:
        print("Not enough data to generate rankings.")
        return

    # Update the stock names back to the DataFrame
    #update_tickers_with_names(rank_stock, 'Ticker')
    #update_tickers_with_names(rank_indust, 'Tickers')

    # Remove the '.TW' or '.TWO' suffix
    remove_ticker_suffix(rank_stock, 'Ticker')
    remove_ticker_suffix(rank_indust, 'Tickers')

    # Save to CSV
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    print("\n\n***")
    today = datetime.now().strftime('%Y%m%d')
    for table, kind in zip([rank_stock, rank_indust],
                           ['stocks', 'industries']):
        filename = f'{code}_{kind}_{period}_{today}.csv'
        table.to_csv(os.path.join(out_dir, filename), index=False)
        print(f'Your "{filename}" is in the "{out_dir}" folder.')
    print("***\n")

    return rank_stock, rank_indust

### Usage Explanation

##### Parameters
source (選擇要分析的股票來源):
- The source of stocks to analyze

min_percentile (最小百分位)
- The minimum percentile for a stock to be included in the rankings.

period (歷史資料時間範圍)：
- The period for which to fetch historical data.

### Error Messages

In [51]:
display(output)

Output()

### RS Rating and Ranking

In [52]:
source = "All Indices" #@param ["S&P 500", "Dow Jones Industrial Average", "NASDAQ 100", "PHLX Semiconductor", "All Indices"]
period = "2y" # @param ["6mo","1y","ytd","2y"]

code_from_name = {
    'S&P 500': 'SPX',
    'Dow Jones Industrial Average': 'DJIA',
    'NASDAQ 100': 'NDX',
    'PHLX Semiconductor': 'SOX',
    'All Indices': 'SPX+DJIA+NDX+SOX',
}

rank_stock, rank_indust = rank(code_from_name[source], period)
display(rank_stock)
display(rank_indust)



***
Your "SPX+DJIA+NDX+SOX_stocks_2y_20240809.csv" is in the "out" folder.
Your "SPX+DJIA+NDX+SOX_industries_2y_20240809.csv" is in the "out" folder.
***



,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
400,VST,Utilities,Utilities - Independent Power Producers,156.88,192.00,238.58,140.22,99,99,99,97,1
141,NVDA,Technology,Semiconductors,153.47,187.10,169.40,177.27,99,99,99,99,2
81,IRM,Real Estate,REIT - Specialty,145.72,124.08,116.53,103.70,99,95,89,69,3
316,FICO,Technology,Software - Application,143.97,131.59,119.32,131.11,99,97,91,96,4
242,GDDY,Technology,Software - Infrastructure,141.55,127.27,132.61,118.52,99,97,97,92,5
...,...,...,...,...,...,...,...,...,...,...,...,...
403,ALB,Basic Materials,Specialty Chemicals,56.02,55.55,87.38,63.10,0,0,19,0,517
470,DXCM,Healthcare,Medical Devices,56.01,80.24,97.70,92.39,0,15,49,41,518
474,INTC,Technology,Semiconductors,54.40,77.54,73.35,113.24,0,10,3,87,519
307,WBA,Healthcare,Pharmaceutical Retailers,48.64,46.14,68.46,74.30,0,0,0,4,520


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
0,Utilities - Independent Power Producers,Utilities,146.16,163.82,205.05,128.39,"VST,NRG",99,99,99,98,1
60,Medical Care Facilities,Healthcare,120.70,101.34,109.72,107.30,"UHS,HCA,DVA",98,90,93,83,2
84,Oil & Gas Midstream,Energy,117.18,108.45,111.29,96.99,"TRGP,OKE,KMI,WMB",97,96,95,50,3
39,Conglomerates,Industrials,114.84,99.05,102.60,85.44,"MMM,HON",96,75,66,13,4
35,REIT - Healthcare Facilities,Real Estate,114.56,99.87,99.82,87.93,"VTR,WELL,DOC",95,82,60,21,5
...,...,...,...,...,...,...,...,...,...,...,...,...
37,Oil & Gas Equipment & Services,Energy,84.67,79.17,91.61,81.75,"BKR,SLB,HAL",4,5,18,5,90
63,Beverages - Brewers,Consumer Defensive,83.57,77.74,87.04,90.99,"STZ,TAP",3,3,5,27,91
17,Auto Parts,Consumer Cyclical,79.66,75.35,87.89,77.96,"GPC,BWA,LKQ,APTV",2,0,6,2,92
45,Airlines,Industrials,77.38,78.44,102.88,91.48,"DAL,LUV,UAL,AAL",1,4,68,29,93


In [53]:
# @title Print Top Percentile Stocks
min_percentile = 90 # @param {"type":"slider","min":1,"max":99,"step":1}
top_stocks = rank_stock[rank_stock[ibd.TITLE_PERCENTILE] >= min_percentile]
num_rows, _ = top_stocks.shape
print(f'\nnumber of filtered tickers: {num_rows}')
top_stock_list = list(top_stocks["Ticker"])
print(top_stock_list)


number of filtered tickers: 52
['VST', 'NVDA', 'IRM', 'FICO', 'GDDY', 'AXON', 'HWM', 'NRG', 'MMM', 'TRGP', 'KKR', 'UHS', 'TSM', 'GE', 'MPWR', 'ANET', 'MHK', 'ISRG', 'AVGO', 'NTAP', 'RCL', 'TYL', 'K', 'VTR', 'EFX', 'CBRE', 'CEG', 'BRO', 'META', 'CFG', 'WELL', 'GRMN', 'HCA', 'COST', 'GS', 'EXR', 'LLY', 'COHR', 'MSI', 'CTAS', 'DHI', 'KLAC', 'GEN', 'TMUS', 'PGR', 'HIG', 'IP', 'FITB', 'EBAY', 'ICE', 'DVA', 'TT']


In [54]:
# @title Filtered Stocks with Increasing RS > 100
filtered_rank_stock = filter_increasing_relative_strength(rank_stock)
display(filtered_rank_stock)

,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
81,IRM,Real Estate,REIT - Specialty,145.72,124.08,116.53,103.70,99,95,89,69,3
454,TRGP,Energy,Oil & Gas Midstream,132.60,125.76,122.31,106.44,98,96,93,75,10
421,TYL,Technology,Software - Application,121.06,108.40,102.57,99.21,95,87,65,57,22
78,VTR,Real Estate,REIT - Healthcare Facilities,118.93,101.50,96.67,85.34,95,74,47,20,24
60,WELL,Real Estate,REIT - Healthcare Facilities,116.95,103.73,103.56,100.23,94,79,68,60,31
339,EXR,Real Estate,REIT - Industrial,116.35,99.11,97.98,90.57,93,68,50,36,36
516,MSI,Technology,Communication Equipment,115.84,104.93,103.03,94.63,92,80,66,47,39
269,IP,Consumer Cyclical,Packaging & Containers,114.94,106.32,105.74,91.51,91,83,73,38,47
255,PM,Consumer Defensive,Tobacco,114.20,99.61,97.58,84.85,89,69,48,19,53
154,AFL,Financial Services,Insurance - Life,113.96,102.40,100.54,93.68,89,75,59,44,55


### RS Rating and Ranking for Taiwan Stocks

In [55]:
from vistock import tw

source = "上市+上櫃" #@param ["上市", "上櫃", "上市+上櫃", "興櫃", "全部"]
period = "2y" # @param ["6mo","1y","ytd","2y"]

code_from_name = {
    '上市': 'TWSE',
    '上櫃': 'TPEX',
    '上市+上櫃': 'TWSE+TPEX',
    '興櫃': 'ESB',
    '全部': 'TWSE+TPEX+ESB'
}

tw_stocks, tw_industries = rank(code_from_name[source], period,
     tickers_getter=tw.get_tickers, ref_ticker='^TWII')
display(tw_stocks)
display(tw_industries)



***
Your "TWSE+TPEX_stocks_2y_20240809.csv" is in the "out" folder.
Your "TWSE+TPEX_industries_2y_20240809.csv" is in the "out" folder.
***



,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
1480,6144,Communication Services,Entertainment,450.79,254.34,119.00,83.36,99.0,99,88,21,1
365,1799,Healthcare,Drug Manufacturers—Specialty & Generic,442.29,330.83,208.06,208.31,99.0,99,99,98,2
1254,8374,Industrials,Industrial Distribution,380.29,263.96,83.04,83.13,99.0,99,41,20,3
1353,2365,Technology,Computer Hardware,356.52,282.91,138.81,171.88,99.0,99,94,97,4
1516,4562,Industrials,Specialty Industrial Machinery,352.39,302.62,159.95,83.68,99.0,99,96,22,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1448,2740,Consumer Cyclical,Restaurants,NaN,54.67,68.02,78.92,NaN,0,6,11,2004
1656,00625K,Unknown,Unknown,NaN,74.13,76.53,80.47,NaN,23,20,14,2005
1794,5906,Consumer Cyclical,Apparel Manufacturing,NaN,70.95,94.46,104.18,NaN,14,65,74,2006
1858,2836A,Financial Services,Banks—Regional,NaN,75.31,79.58,85.47,NaN,28,29,28,2007


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
62,Industrial Distribution,Industrials,159.06,129.66,84.96,88.22,"8374,911608,3114,2373",99.0,98,37,30,1
2,Real Estate—Development,Real Estate,135.00,120.48,125.54,105.70,"2524,5508,5455,2537,3188,1436,2718,4907,5206,3...",98.0,96,99,88,2
37,Entertainment,Communication Services,131.35,101.29,93.90,96.39,"6144,4806,6596,8446,6184,8450,6464,6625,6856",97.0,87,72,72,3
13,Real Estate—Diversified,Real Estate,126.17,119.08,124.72,115.89,"1438,9946,6219,2520,2545,5512,2547",96.0,95,98,97,4
79,Utilities—Renewable,Utilities,120.41,127.74,102.64,106.08,"6869,6873,6806,8087",95.0,97,90,89,5
...,...,...,...,...,...,...,...,...,...,...,...,...
17,Restaurants,Consumer Cyclical,NaN,70.07,76.41,87.20,"3522,1268,2726,2752,2754,2723,2740,1259,2729,2...",NaN,3,9,24,92
29,Electronic Gaming & Multimedia,Communication Services,NaN,100.84,101.51,101.49,"3064,3293,3086,4994,7584,6482,3546,4946,6542,5...",NaN,86,88,76,93
40,Biotechnology,Healthcare,NaN,78.77,79.38,89.32,"6535,4726,6662,1777,4167,6236,4728,4131,1734,4...",NaN,30,13,35,94
43,Apparel Manufacturing,Consumer Cyclical,NaN,81.81,88.99,95.29,"8932,2924,1473,1477,1315,4413,4432,4438,1476,4...",NaN,41,52,68,95


In [56]:
# @title Print Top Percentile Taiwan Stocks
min_percentile = 95 # @param {"type":"slider","min":1,"max":99,"step":1}

top_stocks = tw_stocks[tw_stocks[ibd.TITLE_PERCENTILE] >= min_percentile]
num_rows, _ = top_stocks.shape
print(f'\nnumber of filtered tickers: {num_rows}')
top_stock_list = list(top_stocks["Ticker"])
top_stock_list = [tw.stock_name(ticker) for ticker in top_stock_list]
print(top_stock_list)


number of filtered tickers: 100
['得利影', '易威', '羅昇', '昆盈', '穎漢', '均華', '光聖', '晶彩科', '皇昌', '新復興', '所羅門', '福裕', '康全電訊', '海悅', '欣巴巴', '福大', '慧友', '京城', '弘憶股', '新門', '彬台', '世紀', '全譜', '翔耀', '天品', '福懋油', '花王', '東捷', '慶騰', '弘塑', '志聖', '訊舟', '永信建', '均豪', '鑫科', '高鋒', '安國', '擎亞', '京晨科', '系微', '三地開發', '及成', '天揚', '天方能源', '喬福', '太普高', '順藥', '昇益', '和椿', '聯上發', '藥華藥', '泰偉', '昇陽半導體', '德晉', '鑫龍騰', '旺矽', '訊聯基因', '聯鈞', '華友聯', '藝舍-KY', '鏵友益', '錦明', '鈊象', '泰金-KY', '盟立', '錸德', '華城', '晶悅', '雲豹能源', '惠特', '鈺邦', '宏碩系統', '波力-KY', '富宇', '天剛', '坤悅', '益登', '華義', '森寶', '愛山林', '勝昱', '合騏', '大城地產', '直得', '大飲', '信立', '佳能', '迎廣', '達能', '泰金寶-DR', '宏旭-KY', '加捷生醫', '三發地產', '精材', '精湛', '訊聯', '理銘', '友威科', '德律', '萬潤']


In [57]:
# @title Filtered Taiwan Stocks with Increasing RS > 100

filtered_tw_stocks = filter_increasing_relative_strength(tw_stocks)
update_tickers_with_names(filtered_tw_stocks, 'Ticker')
display(filtered_tw_stocks)

filtered_tw_industries = filter_increasing_relative_strength(tw_industries)
update_tickers_with_names(filtered_tw_industries, 'Tickers')
display(filtered_tw_industries)

,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
1480,得利影,Communication Services,Entertainment,450.79,254.34,119.00,83.36,99.0,99,88,21,1
1516,穎漢,Industrials,Specialty Industrial Machinery,352.39,302.62,159.95,83.68,99.0,99,96,22,5
413,均華,Technology,Semiconductors,328.59,306.87,231.50,162.32,99.0,99,99,96,6
961,海悅,Real Estate,Real Estate Services,275.40,215.50,211.95,122.17,99.0,98,99,88,14
1671,欣巴巴,Industrials,Engineering & Construction,259.48,154.83,119.56,109.68,99.0,96,89,79,15
...,...,...,...,...,...,...,...,...,...,...,...,...
1352,炎洲,Basic Materials,Specialty Chemicals,102.49,96.85,95.74,93.20,78.0,72,67,54,434
607,美吉吉-KY,Industrials,Building Products & Equipment,102.44,98.96,97.42,96.84,78.0,75,70,61,436
857,旭隼,Industrials,Electrical Equipment & Parts,101.61,86.96,86.51,77.62,77.0,58,49,9,452
1763,潤泰全,Consumer Cyclical,Textile Manufacturing,101.56,98.93,94.66,88.12,77.0,75,66,38,453


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
28,Specialty Industrial Machinery,Industrials,118.56,109.43,104.35,102.50,"穎漢,福裕,彬台,志聖,高鋒,喬福,太普高,和椿,盟立,惠特,精湛,友威科,萬潤,信紘科,台...",92.0,93,92,81,7
87,Specialty Business Services,Industrials,105.68,101.87,98.83,92.97,"花王,沈氏,白紗科,秋雨,關貿,信實,政伸,遠雄港,耕興",87.0,88,82,61,11
